In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import sys
import os
from subprocess import call 
import pathlib

In [4]:
import numpy as np
import pandas as pd
import xarray as xr

In [5]:
from dask.distributed import Client
from dask.diagnostics import ProgressBar

### parameters for papermill

In [6]:
region = 'NZ'
ndays = 30
roll = 15
climatology = [1991, 2020]
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]

### create cluster

In [7]:
client = Client(
    n_workers=6, threads_per_worker=4, memory_limit="8GB", local_directory="./dask"
)

In [8]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 44.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39835,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 44.70 GiB
Comm: tcp://127.0.0.1:45129,Total threads: 4
Dashboard: http://127.0.0.1:38855/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:34859,


In [9]:
sys.path.append('../code/')

In [10]:
import src

In [11]:
opath = pathlib.Path(f'/media/nicolasf/END19101/data/OISST/daily/{region}')

In [12]:
lfiles = list(opath.glob("sst.day.mean.????.v2.nc"))

In [13]:
lfiles.sort()

In [14]:
lfiles[0]

PosixPath('/media/nicolasf/END19101/data/OISST/daily/NZ/sst.day.mean.1981.v2.nc')

In [15]:
lfiles[-1]

PosixPath('/media/nicolasf/END19101/data/OISST/daily/NZ/sst.day.mean.2022.v2.nc')

In [16]:
dset = xr.open_mfdataset(lfiles, parallel=True, combine_attrs="drop_conflicts", compat="override", coords=['time'])

In [17]:
dset

<xarray.Dataset>
Dimensions:  (time: 14952, lon: 72, lat: 80)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2022-08-08
  * lon      (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
  * lat      (lat) float32 -49.88 -49.62 -49.38 -49.12 ... -30.62 -30.38 -30.12
Data variables:
    sst      (time, lat, lon) float32 dask.array<chunksize=(122, 80, 72), meta=np.ndarray>

### need to start a first of January 

In [18]:
# dset = dset.sel(time=slice('1982-01-01',None))

### if regions is 'Ninos' or 'IOD', then first calculates the regional averages

In [19]:
if region == 'Ninos':
    
    dset['sst'] = src.calculates_ninos(dset['sst'], nino='all')
    
elif region == 'IOD': 
    
    dset['sst']= src.calculates_IOD_nodes(dset['sst'], IOD_node='all')

In [20]:
dset

<xarray.Dataset>
Dimensions:  (time: 14952, lon: 72, lat: 80)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2022-08-08
  * lon      (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
  * lat      (lat) float32 -49.88 -49.62 -49.38 -49.12 ... -30.62 -30.38 -30.12
Data variables:
    sst      (time, lat, lon) float32 dask.array<chunksize=(122, 80, 72), meta=np.ndarray>

### remove the 29th of Feb

In [21]:
standard_calendar = dset.time

In [22]:
dset = dset.convert_calendar('noleap')

### calculates the rolling averages if needed 

In [23]:
if ndays > 1: 
    
    dset['sst'] = dset['sst'].rolling({'time':ndays}, center=False, min_periods=ndays).mean('time')
    
    dset = dset.isel(time=slice(ndays+1,None))

In [24]:
dset

<xarray.Dataset>
Dimensions:  (time: 14911, lat: 80, lon: 72)
Coordinates:
  * time     (time) object 1981-10-02 00:00:00 ... 2022-08-08 00:00:00
  * lon      (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
  * lat      (lat) float32 -49.88 -49.62 -49.38 -49.12 ... -30.62 -30.38 -30.12
Data variables:
    sst      (time, lat, lon) float64 dask.array<chunksize=(120, 80, 72), meta=np.ndarray>

### now expands the dataset along a dummy dimension 

In [25]:
dset_roll = dset[['sst']].copy()

In [26]:
dset_roll = dset_roll.rolling({'time':roll}, center=True, min_periods=roll).construct(window_dim='roll')

### selects the climatological period 

In [27]:
clim = dset_roll.sel(time=slice(str(climatology[0]), str(climatology[1])))

In [28]:
clim

,Array,Chunk
Bytes,7.05 GiB,240.60 MiB
Shape,"(10950, 80, 72, 15)","(365, 80, 72, 15)"
Count,1766 Tasks,31 Chunks
Type,float64,numpy.ndarray


### transpose, then rechunks 

In [29]:
if region == 'Ninos': 
    
    clim = clim.transpose(*['time','roll','nino'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'nino':len(clim['nino'])})
    
elif region == 'IOD': 

    clim = clim.transpose(*['time','roll','IOD'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'IOD':len(clim['IOD'])})
    
else: 
    
    clim = clim.transpose(*['time','roll','lat','lon'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'lat':2, 'lon':20})

In [30]:
clim

,Array,Chunk
Bytes,7.05 GiB,50.13 MiB
Shape,"(10950, 15, 80, 72)","(10950, 15, 2, 20)"
Count,3001 Tasks,160 Chunks
Type,float64,numpy.ndarray


In [31]:
clim_q = clim.groupby(clim.time.dt.dayofyear).quantile(quantiles, dim=['time','roll'])

In [32]:
clim_ave = clim.groupby(clim.time.dt.dayofyear).mean(dim=['time','roll'])

In [33]:
clim_std = clim.groupby(clim.time.dt.dayofyear).std(dim=['time','roll'])

In [34]:
with ProgressBar(): 
    clim_q = clim_q.compute()
    clim_ave = clim_ave.compute()
    clim_std = clim_std.compute()

In [35]:
clim_q = clim_q.rename({'sst':'quantiles'})

In [36]:
clim_ave

<xarray.Dataset>
Dimensions:    (lat: 80, lon: 72, dayofyear: 365)
Coordinates:
  * lon        (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
  * lat        (lat) float32 -49.88 -49.62 -49.38 ... -30.62 -30.38 -30.12
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Data variables:
    sst        (lat, lon, dayofyear) float64 9.787 9.813 9.839 ... 21.67 21.71

In [37]:
clim_q['average'] = clim_ave['sst']

In [38]:
clim_q

<xarray.Dataset>
Dimensions:    (lon: 72, lat: 80, dayofyear: 365, quantile: 5)
Coordinates:
  * lon        (lon) float32 162.1 162.4 162.6 162.9 ... 179.1 179.4 179.6 179.9
  * lat        (lat) float32 -49.88 -49.62 -49.38 ... -30.62 -30.38 -30.12
  * quantile   (quantile) float64 0.1 0.25 0.5 0.75 0.9
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Data variables:
    quantiles  (dayofyear, quantile, lat, lon) float64 9.088 9.131 ... 22.69
    average    (lat, lon, dayofyear) float64 9.787 9.813 9.839 ... 21.67 21.71

In [39]:
clim_q['std'] = clim_std['sst']

In [40]:
opath = pathlib.Path(f'../outputs/{region}')

In [41]:
opath.mkdir(exist_ok=True)

### saves to ZARR

In [42]:
clim_q.to_zarr(opath.joinpath(f'{region}_OISST_{ndays}days_climatology_{roll}_window.zarr'))